In [28]:
import requests
import json
import csv

In [48]:
database = []

In [49]:
websites = [
    'tablas-creek.db.wine/_next/data/txRWYFogfzAsi7hSbsq7l',
    'bodega-furlotti.db.wine/_next/data/NmH1SDt-7v4D94KNGDHJj',
    'bodega-weinert.db.wine/_next/data/n6G2s6LWbyd5X5zcsdE9K'
    'bodega-y-vinedos-catena.db.wine/_next/data/VHFggATMDxW8UpMG4SE3X',
    'di-giovanna.db.wine/_next/data/ejV5bkc6W5D1xHJ4TIJYu'
    'chateau-capion.db.wine/_next/data/poXS7OMLewC47LDWpd-N6',
    'paradise-rescued.db.wine/_next/data/ypCl9yCqOMMTHo5btVKIe',
    'epoch-estate-wines.db.wine/_next/data/eH-Kz918FmLFXZtaNt1pQ',
    'eight-at-the-gate-wines.db.wine/_next/data/GreAOcA1RMgzmgrHNZIhx',
    'miner-family-winery.db.wine/_next/data/cNJYZ-LwuqOMy9BcNn29H',
]


for website in websites:
    url = 'https://' + website + '/wines.json'
    response = requests.get(url)

    if response.status_code == 200:
        wine_ids = []
        # print(response.json())
        wines = response.json()['pageProps']['wines']
        for wine in wines:
            print("Working on", wine['name'], str(wine['id']))
            wine_ids.append(wine['id'])
            wine_response = requests.get('https://' + website + '/wines/' + str(wine['id']) + '.json')
            if wine_response.status_code == 200:
                wine = wine_response.json()['pageProps']
                for vintage in wine['vintages']:
                    vintage['name'] = wine['wine']['name']
                    vintage['description'] = wine['wine']['description']
                    vintage['origin'] = wine['winery']['location']
                    database.append(vintage)
        print(wine_ids)


Working on Esprit de Tablas WN-IFCYSWTHK
Working on Cinsaut WN-TFKQASGLH
Working on Esprit de Tablas Blanc WN-YFBGDJTXH
Working on Patelin de Tablas Rosé WN-ABOCLSIQN
Working on Patelin de Tablas WN-VGWFTKBCP
Working on Cotes de Tablas WN-KSIBVPDZE
Working on Patelin de Tablas Blanc WN-ZQFKUTHRN
Working on Roussanne WN-RPQZWXGOB
Working on Cotes de Tablas Blanc WN-XLOTEQZDV
Working on Dianthus WN-TFDOAWUJS
Working on Grenache WN-MJIBHELWA
Working on Tannat WN-NQKTXZMUO
Working on Vaccarese WN-BTGWNPURF
['WN-IFCYSWTHK', 'WN-TFKQASGLH', 'WN-YFBGDJTXH', 'WN-ABOCLSIQN', 'WN-VGWFTKBCP', 'WN-KSIBVPDZE', 'WN-ZQFKUTHRN', 'WN-RPQZWXGOB', 'WN-XLOTEQZDV', 'WN-TFDOAWUJS', 'WN-MJIBHELWA', 'WN-NQKTXZMUO', 'WN-BTGWNPURF']
Working on ROSADO DE MALBEC WN-DVXRTENWZ
Working on FAMILIA FURLOTTI   FINCA PARAJE ALTAMIRA WN-LZXDQPEBK
Working on Criolla WN-VSNXMAUYD
Working on Familia Furlotti Malbec WN-FCBNUARKY
Working on Familia Furlotti Pinot Noir WN-GWRTECPYZ
Working on FAMILIA FURLOTTI  FINCA LUNLUNTA W

In [50]:
print(len(database))

218


In [51]:
# save database to file
with open('database-everything.json', 'w') as outfile:
    json.dump(database, outfile)

In [61]:
# cleanup
clean_database = []
for wine in database:
  if wine['abv'] or wine['acidity'] or wine['ph'] or wine['sugar']:
    print(wine['origin'])
    new_wine = {
        "name": wine['name'],
        "vintage-name": wine['appellation']['name'],
        "origin": wine['origin'][0]['name'],
        "description": wine['description'],
        "year": wine['year'],
        "bottle-shot": wine['bottleShot'],
        "abv": wine['abv'],
        "acidity": wine['acidity'],
        "ph": wine['ph'],
        "sugar": wine['sugar'],
        "currency": wine['currency'],
        "price": wine['price']
    }
    if len(wine['origin']) > 1:
        new_wine['origin'] = wine['origin'][1]['name'] +  ", " + wine['origin'][0]['name']
    clean_database.append(new_wine)

[{'__typename': 'Location', 'name': 'United States'}, {'__typename': 'Location', 'name': 'California'}]
[{'__typename': 'Location', 'name': 'United States'}, {'__typename': 'Location', 'name': 'California'}]
[{'__typename': 'Location', 'name': 'United States'}, {'__typename': 'Location', 'name': 'California'}]
[{'__typename': 'Location', 'name': 'United States'}, {'__typename': 'Location', 'name': 'California'}]
[{'__typename': 'Location', 'name': 'United States'}, {'__typename': 'Location', 'name': 'California'}]
[{'__typename': 'Location', 'name': 'United States'}, {'__typename': 'Location', 'name': 'California'}]
[{'__typename': 'Location', 'name': 'United States'}, {'__typename': 'Location', 'name': 'California'}]
[{'__typename': 'Location', 'name': 'United States'}, {'__typename': 'Location', 'name': 'California'}]
[{'__typename': 'Location', 'name': 'United States'}, {'__typename': 'Location', 'name': 'California'}]
[{'__typename': 'Location', 'name': 'United States'}, {'__typena

In [59]:
# cleanup and csv
columns = ["name", "vintage-name", "origin", "description", "year", "bottle-shot", "abv", "acidity", "ph", "sugar", "currency", "price"]

with open('wines.csv', 'w', newline='') as outfile:
    writer = csv.DictWriter(outfile, fieldnames=columns)
    writer.writeheader()
    for wine in database:
        if wine['abv'] or wine['acidity'] or wine['ph'] or wine['sugar']:
            print(wine['origin'])
            new_wine = {
                "name": wine['name'],
                "vintage-name": wine['appellation']['name'],
                "origin": wine['origin'][0]['name'],
                "description": wine['description'],
                "year": wine['year'],
                "bottle-shot": wine['bottleShot'],
                "abv": wine['abv'],
                "acidity": wine['acidity'],
                "ph": wine['ph'],
                "sugar": wine['sugar'],
                "currency": wine['currency'],
                "price": wine['price']
            }
            if len(wine['origin']) > 1:
                new_wine['origin'] = wine['origin'][1]['name'] +  ", " + wine['origin'][0]['name']
            writer.writerow(new_wine)



[{'__typename': 'Location', 'name': 'United States'}, {'__typename': 'Location', 'name': 'California'}]
[{'__typename': 'Location', 'name': 'United States'}, {'__typename': 'Location', 'name': 'California'}]
[{'__typename': 'Location', 'name': 'United States'}, {'__typename': 'Location', 'name': 'California'}]
[{'__typename': 'Location', 'name': 'United States'}, {'__typename': 'Location', 'name': 'California'}]
[{'__typename': 'Location', 'name': 'United States'}, {'__typename': 'Location', 'name': 'California'}]
[{'__typename': 'Location', 'name': 'United States'}, {'__typename': 'Location', 'name': 'California'}]
[{'__typename': 'Location', 'name': 'United States'}, {'__typename': 'Location', 'name': 'California'}]
[{'__typename': 'Location', 'name': 'United States'}, {'__typename': 'Location', 'name': 'California'}]
[{'__typename': 'Location', 'name': 'United States'}, {'__typename': 'Location', 'name': 'California'}]
[{'__typename': 'Location', 'name': 'United States'}, {'__typena

In [62]:
# save as json
with open('wines.json', 'w') as outfile:
    json.dump(clean_database, outfile)

In [ ]:
# with pandas
import pandas as pd
df = pd.DataFrame(database)
df.to_csv('wines.csv', index=False)